<a href="https://colab.research.google.com/github/faranbutt/ETL_Banks_Project/blob/main/banks_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install numpy
!pip install beautifulsoup4
!pip install requests
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3
from datetime import datetime
import requests

In [161]:
output_file = 'Largest_banks_data.csv'
db_out = 'Banks.db'
log_file = 'code_log.txt'
table_name = 'Largest_banks'
url = 'https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks'
df  = pd.DataFrame(columns=['Name','MC_USD_Billion',"MC_EUR_Billion","MC_GBP_Billion","MC_INR_Billion"])
df

,Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion


In [162]:
r = requests.get(url).text

In [163]:
def log_progress(message):
  timestamp_format = '%Y-%d-%h-%H:%M:%S'
  now = datetime.now()
  timestamp =  now.strftime(timestamp_format)
  with open(log_file,'a') as log:
    log.write(f'{timestamp} : {message} \n')

In [223]:
def extract(df):
  data = BeautifulSoup(r,'html.parser')
  tables = data.find_all('table')
  rows = tables[0].find_all('tr')
  for row in rows:
    cols = row.find_all('td')
    if len(cols) != 0:
      data_dict = {
          "Name" : cols[1].find_all('a')[1].text,
          "MC_USD_Billion" : float(cols[2].text)
      }

      df1 = pd.DataFrame(data_dict,index=[0])
      df = pd.concat([df,df1],ignore_index=True)
  return df

In [236]:
extracted_data = extract(df)
cols[2].contents[0][:-1]

'136.81'

In [239]:
def transform(data):
  USD_data = data['MC_USD_Billion'].to_list()
  ER = pd.read_csv('exchange_rate.csv')
  count = 0
  for i in range(0,3):
    currency = ER.iloc[i][0]
    rate = ER.iloc[i][1]
    d = [round(x*rate,2) for x in USD_data]
    if count == 0:
      data['MC_EUR_Billion'] = d
    elif count == 1:
      data['MC_GBP_Billion'] = d
    elif count == 2:
      data['MC_INR_Billion'] = d
    count +=1
  return data

In [240]:
transformed_data = transform(extracted_data)
transformed_data

,Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,Bank of America,231.52,215.31,185.22,19204.58
2,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,HDFC Bank,157.91,146.86,126.33,13098.63
5,Wells Fargo,155.87,144.96,124.70,12929.42
6,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
7,Morgan Stanley,140.83,130.97,112.66,11681.85
8,China Construction Bank,139.82,130.03,111.86,11598.07
9,Bank of China,136.81,127.23,109.45,11348.39


In [212]:
def load_to_csv(data,output_file):
  data.to_csv(output_file)
def load_to_db(data,conn,table_name):
  data.to_sql(table_name,con=conn,if_exists='replace',index=False)

In [213]:
load_to_csv(transformed_data,output_file)
load_to_db(transformed_data,conn,table_name)

In [214]:
def run_queries(query,conn):
  sql_data = pd.read_sql(query,con=conn)
  return sql_data

In [221]:
sql_query1 = 'SELECT * FROM Largest_banks'
sql_query2 = 'SELECT AVG(MC_GBP_Billion) FROM Largest_banks'
sql_query3 = 'SELECT Name from Largest_banks LIMIT 5'
print(f"QUERY = {sql_query1} \n")
print(run_queries(sql_query1,conn))
print('\n ****************************************************************************************************************** \n')
print(f"QUERY = {sql_query2} \n")
print(run_queries(sql_query2,conn))
print(' \n ****************************************************************************************************************** \n')
print(f"QUERY = {sql_query3} \n")
print(run_queries(sql_query3,conn))

QUERY = SELECT * FROM Largest_banks 

                                      Name  MC_USD_Billion  MC_EUR_Billion  \
0                           JPMorgan Chase          432.92          402.62   
1                          Bank of America          231.52          215.31   
2  Industrial and Commercial Bank of China          194.56          180.94   
3               Agricultural Bank of China          160.68          149.43   
4                                HDFC Bank          157.91          146.86   
5                              Wells Fargo          155.87          144.96   
6                        HSBC Holdings PLC          148.90          138.48   
7                           Morgan Stanley          140.83          130.97   
8                  China Construction Bank          139.82          130.03   
9                            Bank of China          136.81          127.23   

   MC_GBP_Billion  MC_INR_Billion  
0          346.34        35910.71  
1          185.22        19204.

In [225]:
log_progress('Preliminaries complete. Initiating ETL process')
extracted_data = extract(df)
log_progress('Data extraction complete. Initiating Transformation process')
transformed_data = transform(extracted_data)
log_progress('Data transformation complete. Initiating Loading process')
load_to_csv(transformed_data,output_file)
log_progress('Data saved to CSV file')
conn = sqlite3.connect(db_out)
log_progress('SQL Connection initiated')
load_to_db(transformed_data,conn,table_name)
log_progress('Data loaded to Database as a table, Executing queries')
log_progress('Process Complete')
conn.close()
log_progress('Server Connection closed')